In [1]:
from src.data.classes import NdData

In [1]:
from src.api import DataAPI , Predictor

DataAPI.update()
DataAPI.prepare_train_data()
Predictor.update_factors()

Update Files
Mon Aug 12 22:43:06 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.04 Secs
Mon Aug 12 22:43:06 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Mon Aug 12 22:43:06 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.02 Secs
Mon Aug 12 22:43:09 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 2.37 Secs
Mon Aug 12 22:43:13 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 4.58 Secs
Mon Aug 12 22:43:15 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240808.feather Done! Cost 0.10 Secs
Mon Aug 12 22:43:15 2024 : Updated ~ data\DataBase\DB_models\longcl_exp\2024\longcl_exp.20240809.feather Done! Cost 0.08 Secs
Mon Aug 12 22:43:15 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240812.feather Done! Cost 0.04 Secs
Mon Aug 12 22:43:15 2024 : Updated ~ data\DataBase\DB_models\risk_cov\2024\risk_cov.20240812.fea

In [2]:
from src.api import DataAPI , Predictor
DataAPI.prepare_train_data()
Predictor.update_factors()

predict is True , Data Processing start!
6 datas :['y', 'day', '30m', 'style', 'indus', 'week']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 1.54 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 1.00 secs
 --> labels blocks merging (2)...... finished! Cost 0.20 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 3.87 secs
y blocks loading finished! Cost 7.08 secs
y blocks process...... finished! Cost 6.68 secs
y blocks masking...... finished! Cost 0.16 secs
y blocks saving ...... finished! Cost 0.19 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 14.26 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 1.49 secs
day blocks loading finished! Cost 1.53 secs
day blocks process...... finished! Cost 0.28 secs
day blocks masking...... finished! Cost 0.20

In [2]:
from src.api import ModelTestor
ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
